Name: Rajkumar  Chandrasekaran |                                    
Roll No : G24AI2078 |                                           
Email : g24ai2078@iitj.ac.in 

In [ ]:
from sqlite3 import dbapi2 as sqlite3
print(sqlite3.sqlite_version_info)

(3, 37, 2)


In [ ]:
import pandas as pd
from IPython.display import display, HTML

In [ ]:
dbname = "music_streaming_1.db"

def printSqlResults(cursor, tbl):
  df=pd.DataFrame(cursor.fetchall(), columns=[i[0] for i in cursor.description])
  display(HTML("<b><font color=Green>"+tbl+"</font></b>"+df.to_html(index=False)))

def runSql(caption, query):
  conn = sqlite3.connect(dbname)
  cursor=conn.cursor()
  cursor.execute(query)
  printSqlResults(cursor,caption)
  conn.close()

In [ ]:
conn = sqlite3.connect(dbname)
cursor = conn.cursor()

cursor.execute("""
      CREATE TABLE IF NOT EXISTS Users (
        user_id INTEGER PRIMARY KEY,
        name VARCHAR(100) NOT NULL,
        email VARCHAR(100) NOT NULL UNIQUE
      );
""")

cursor.execute("""
  CREATE TABLE IF NOT EXISTS Songs (
    song_id INTEGER PRIMARY KEY,
    title VARCHAR(100) NOT NULL,
    artist VARCHAR(100) NOT NULL,
    genre VARCHAR(100)
  );
"""
)

cursor.execute("""
    CREATE TABLE IF NOT EXISTS Listens (
      listen_id INTEGER PRIMARY KEY,
      user_id INTEGER NOT NULL,
      song_id INTEGER NOT NULL,
      rating FLOAT,
      listen_time TIMESTAMP,
      FOREIGN KEY (user_id) REFERENCES Users(user_id),
      FOREIGN KEY (song_id) REFERENCES Songs(song_id)
    );
""")

cursor.execute("""
  CREATE TABLE IF NOT EXISTS Recommendations (
      recommendation_id INTEGER NOT NULL,
      recommendation_time TIMESTAMP,
      user_id INTEGER NOT NULL,
      song_id INTEGER NOT NULL,
      FOREIGN KEY (user_id) REFERENCES Users(user_id),
      FOREIGN KEY (song_id) REFERENCES Songs(song_id)
  );
""")

conn.commit()
conn.close()

In [ ]:
conn = sqlite3.connect(dbname)
sqlite3.enable_callback_tracebacks(True)

cursor=conn.cursor()
cursor.execute("delete from Songs;")
cursor.execute("delete from Users;")
cursor.execute("delete from Listens;")
cursor.execute("delete from Recommendations;")

#Insert data into the table

cursor.execute("""
  INSERT INTO Users (user_id, name, email)
  VALUES
    (1, 'Mickey', 'mickey@example.com'),
    (2, 'Minnie', 'minnie@example.com'),
    (3, 'Daffy', 'daffy@example.com'),
    (4, 'Pluto', 'pluto@example.com');
""")
# Insert sample songs from Taylor Swift, Ed Sheeran, Beatles
cursor.execute("""
INSERT INTO Songs (song_id, title, artist, genre)
VALUES
    (1, 'Evermore', 'Taylor Swift', 'Pop'),
    (2, 'Willow', 'Taylor Swift', 'Pop'),
    (3, 'Shape of You', 'Ed Sheeran', 'Rock'),
    (4, 'Photograph', 'Ed Sheeran', 'Rock'),
    (5, 'Shivers', 'Ed Sheeran', 'Rock'),
    (6, 'Yesterday', 'Beatles', 'Classic'),
    (7, 'Yellow Submarine', 'Beatles', 'Classic'),
    (8, 'Hey Jude', 'Beatles', 'Classic'),
    (9, 'Bad Blood', 'Taylor Swift', 'Rock'),
    (10, 'DJ Mix', 'DJ', NULL);
""")

# Insert sample listens
cursor.execute("""
INSERT INTO Listens (listen_id, user_id, song_id, rating, listen_time)
VALUES
    (1, 1, 1, 4.5, '2024-08-30 14:35:00'),
    (2, 1, 2, 4.2, NULL),
    (3, 1, 6, 3.9, '2024-08-29 10:15:00'),
    (4, 2, 2, 4.7, NULL),
    (5, 2, 7, 4.6, '2024-08-28 09:20:00'),
    (6, 2, 8, 3.9, '2024-08-27 16:45:00'),
    (7, 3, 1, 2.9, NULL),
    (8, 3, 2, 4.9, '2024-08-26 12:30:00'),
    (9, 3, 6, NULL, NULL),
    (10, 2, 1, 4.8, '2024-08-30 18:00:00');
""")
# Commit changes and close the connection
conn.commit()
conn.close()

runSql('Users', "select * from Users;")
runSql('Songs', "select * from Songs;")
runSql('Listens', "select * from Listens;")

user_id,name,email
1,Mickey,mickey@example.com
2,Minnie,minnie@example.com
3,Daffy,daffy@example.com
4,Pluto,pluto@example.com


song_id,title,artist,genre
1,Evermore,Taylor Swift,Pop
2,Willow,Taylor Swift,Pop
3,Shape of You,Ed Sheeran,Rock
4,Photograph,Ed Sheeran,Rock
5,Shivers,Ed Sheeran,Rock
6,Yesterday,Beatles,Classic
7,Yellow Submarine,Beatles,Classic
8,Hey Jude,Beatles,Classic
9,Bad Blood,Taylor Swift,Rock
10,DJ Mix,DJ,None


listen_id,user_id,song_id,rating,listen_time
1,1,1,4.5,2024-08-30 14:35:00
2,1,2,4.2,None
3,1,6,3.9,2024-08-29 10:15:00
4,2,2,4.7,None
5,2,7,4.6,2024-08-28 09:20:00
6,2,8,3.9,2024-08-27 16:45:00
7,3,1,2.9,None
8,3,2,4.9,2024-08-26 12:30:00
9,3,6,NaN,None
10,2,1,4.8,2024-08-30 18:00:00


In [ ]:
qry_classic_songs = """
SELECT Songs.title, Songs.artist FROM Songs
WHERE Songs.genre= 'Classic';
"""
runSql('Classic Query', qry_classic_songs)

title,artist
Yesterday,Beatles
Yellow Submarine,Beatles
Hey Jude,Beatles


In [ ]:
#Find titles and artists if songs in Classic Genre with titles starting like Ye...

qry_classic_songs2 = """
SELECT Songs.title, Songs.artist FROM Songs
WHERE Songs.genre= 'Classic' AND
  Songs.title LIKE 'Ye%';
"""
runSql('Classic Query with songs like Ye..', qry_classic_songs2)

title,artist
Yesterday,Beatles
Yellow Submarine,Beatles


In [ ]:
#List all the genres in the Songs table
qry_genres = """
SELECT genre from Songs;
"""
runSql('All genres', qry_genres)

genre
Pop
Pop
Rock
Rock
Rock
Classic
Classic
Classic
Rock
None


In [ ]:
#Find the unique genres from the Songs table

qry_distinct = """
SELECT DISTINCT genre
FROM Songs;
"""

runSql('Distinct Genres', qry_distinct)

genre
Pop
Rock
Classic
None


In [ ]:
#Find the number of songs by Taylor Swift in different genres

qry_taylor_count="""
SELECT artist, genre,count(*) as num_songs
FROM Songs
WHERE Songs.artist='Taylor Swift'
GROUP BY artist, genre;
"""
runSql('Count of songs by Taylor Swift in different genres', qry_taylor_count)

artist,genre,num_songs
Taylor Swift,Pop,2
Taylor Swift,Rock,1


In [ ]:
#Find the number of songs by all artists in different genres

qry_artist_count="""
SELECT artist, genre,count(*) as num_songs
FROM Songs
GROUP BY artist, genre;
"""
runSql('Count of songs by Taylor Swift in different genres', qry_artist_count)


artist,genre,num_songs
Beatles,Classic,3
DJ,None,1
Ed Sheeran,Rock,3
Taylor Swift,Pop,2
Taylor Swift,Rock,1


In [ ]:
#Create one single table for songs and listens. Redundancies are allowed

qry_large_table = """
SELECT * FROM Songs
LEFT JOIN Listens ON Songs.song_id=Listens.song_id
LEFT JOIN Users ON Listens.user_id=Users.user_id;
"""
runSql('One Large Table',qry_large_table)

song_id,title,artist,genre,listen_id,user_id,song_id,rating,listen_time,user_id,name,email
1,Evermore,Taylor Swift,Pop,1.0,1.0,1.0,4.5,2024-08-30 14:35:00,1.0,Mickey,mickey@example.com
1,Evermore,Taylor Swift,Pop,10.0,2.0,1.0,4.8,2024-08-30 18:00:00,2.0,Minnie,minnie@example.com
1,Evermore,Taylor Swift,Pop,7.0,3.0,1.0,2.9,None,3.0,Daffy,daffy@example.com
2,Willow,Taylor Swift,Pop,2.0,1.0,2.0,4.2,None,1.0,Mickey,mickey@example.com
2,Willow,Taylor Swift,Pop,4.0,2.0,2.0,4.7,None,2.0,Minnie,minnie@example.com
2,Willow,Taylor Swift,Pop,8.0,3.0,2.0,4.9,2024-08-26 12:30:00,3.0,Daffy,daffy@example.com
3,Shape of You,Ed Sheeran,Rock,NaN,NaN,NaN,NaN,None,NaN,None,None
4,Photograph,Ed Sheeran,Rock,NaN,NaN,NaN,NaN,None,NaN,None,None
5,Shivers,Ed Sheeran,Rock,NaN,NaN,NaN,NaN,None,NaN,None,None
6,Yesterday,Beatles,Classic,3.0,1.0,6.0,3.9,2024-08-29 10:15:00,1.0,Mickey,mickey@example.com


In [ ]:
#Find the Highly Rates Songs. Consider Rating above 4.6 as highly rated

qry_rating_songs="""
SELECT Songs.song_id, Songs.title, Songs.artist, Listens.rating
FROM Songs
JOIN Listens ON Songs.song_id = Listens.song_id
WHERE Listens.rating > 4.6
"""
runSql('Highly Rated Songs',qry_rating_songs)

#Find tha average rating of every song

qry_avg_songs="""
SELECT Songs.song_id, Songs.title, Songs.artist, AVG(Listens.rating)
FROM Songs
JOIN Listens ON Songs.song_id = Listens.song_id
GROUP BY Songs.song_id, Songs.title, Songs.artist
"""
runSql('Highly Rated Songs',qry_avg_songs)



song_id,title,artist,rating
2,Willow,Taylor Swift,4.7
2,Willow,Taylor Swift,4.9
1,Evermore,Taylor Swift,4.8


song_id,title,artist,AVG(Listens.rating)
1,Evermore,Taylor Swift,4.066667
2,Willow,Taylor Swift,4.600000
6,Yesterday,Beatles,3.900000
7,Yellow Submarine,Beatles,4.600000
8,Hey Jude,Beatles,3.900000


In [ ]:
#Find the popular songs by counting the listens

qry_pop_songs = """
SELECT Songs.song_id, Songs.title, Songs.artist, count(Listens.song_id)
FROM Songs
JOIN Listens
ON Songs.song_id=Listens.song_id
GROUP BY Songs.title, Songs.artist
ORDER BY COUNT(Listens.song_id) DESC;
"""

runSql('Popular Songs', qry_pop_songs)

song_id,title,artist,count(Listens.song_id)
1,Evermore,Taylor Swift,3
2,Willow,Taylor Swift,3
6,Yesterday,Beatles,2
8,Hey Jude,Beatles,1
7,Yellow Submarine,Beatles,1


In [ ]:
#Find songs by Ed Sheeran and Taylor Swift

qry_edtaylor="""
SELECT title, artist
FROM Songs
WHERE artist IN ('Ed Sheeran', 'Taylor Swift');
"""
runSql('Songs by Ed Sheeran and Taylor Swift', qry_edtaylor)

#Find songs from both pop and rock genres

qry_popandrock = """
SELECT title, artist
FROM Songs
WHERE genre='Pop'
UNION
SELECT title, artist
FROM Songs
WHERE genre='Rock'
"""
runSql('Songs on both Pop and Rock', qry_popandrock)


title,artist
Evermore,Taylor Swift
Willow,Taylor Swift
Shape of You,Ed Sheeran
Photograph,Ed Sheeran
Shivers,Ed Sheeran
Bad Blood,Taylor Swift


title,artist
Bad Blood,Taylor Swift
Evermore,Taylor Swift
Photograph,Ed Sheeran
Shape of You,Ed Sheeran
Shivers,Ed Sheeran
Willow,Taylor Swift


In [ ]:
#Find songs listened to by user_id 1

qry_listen_usr="""
SELECT title, artist
FROM Songs
WHERE song_id IN (SELECT song_id FROM Listens WHERE Listens.listen_time IS NULL);
"""
runSql('Songs listened by user_id=1',qry_listen_usr)

title,artist
Evermore,Taylor Swift
Willow,Taylor Swift
Yesterday,Beatles


In [ ]:
#Computing the recommendations
#Find the song pairs which are shared across > 1 user

qry_sharing="""
WITH song_similarity AS (
SELECT u1.song_id as song1, u2.song_id as song2, COUNT(*) AS common_users
FROM LISTENS u1
JOIN LISTENS u2
ON u1.user_id=u2.user_id
AND u1.song_id != u2.song_id
GROUP BY u1.song_id, u2.song_id
HAVING COUNT(*)>1
),

recs AS (
  SELECT user_id, song2 as song_id
  FROM song_similarity
  JOIN Listens as L
  ON L.song_id = song_similarity.song1
  WHERE song_similarity.song2 NOT IN
  (SELECT song_id FROM Listens as temp where temp.user_id=L.user_id)
)
select * from recs;
"""
runSql('Songs shared', qry_sharing)

user_id,song_id
2,6
2,6


#ASSIGNMENT-2. To be submitted as a PDF with queries and screenshots.

1. Insert the above into the recommendations table
2. Generate the recommendaions for Minnie
3. Re-do the generation of recommendations now on the basis of listen time
4. Generate new recommendations
5. What are the differences with the static method on #2 above

In [ ]:
#1 Insert the above into the recommendations table
from sqlite3 import dbapi2 as sqlite3

conn = sqlite3.connect(dbname)
cur = conn.cursor()
cur.execute("PRAGMA foreign_keys = ON;")
cur.execute("PRAGMA journal_mode = WAL;")

sql_q1 = """
INSERT INTO Recommendations (recommendation_id, recommendation_time, user_id, song_id)
WITH song_similarity AS (
  SELECT u1.song_id AS song1,
         u2.song_id AS song2,
         COUNT(*)    AS common_users
  FROM Listens u1
  JOIN Listens u2
    ON u1.user_id = u2.user_id
   AND u1.song_id <> u2.song_id
  GROUP BY u1.song_id, u2.song_id
  HAVING COUNT(*) > 1
),
recs AS (
  SELECT DISTINCT
         L.user_id,
         song_similarity.song2 AS song_id
  FROM song_similarity
  JOIN Listens L
    ON L.song_id = song_similarity.song1
  WHERE song_similarity.song2 NOT IN (
    SELECT song_id FROM Listens temp WHERE temp.user_id = L.user_id
  )
),
maxid AS (
  SELECT COALESCE(MAX(recommendation_id), 0) AS m FROM Recommendations
)
SELECT
  m + ROW_NUMBER() OVER (ORDER BY user_id, song_id) AS recommendation_id,
  CURRENT_TIMESTAMP                                  AS recommendation_time,
  user_id,
  song_id
FROM recs, maxid;
"""

cur.execute(sql_q1)
conn.commit()
conn.close()




In [ ]:
conn = sqlite3.connect(dbname)
cursor = conn.cursor()
cursor.execute("SELECT * FROM Recommendations;")
rows = cursor.fetchall()
import pandas as pd
from IPython.display import display
cols = [desc[0] for desc in cursor.description]
df = pd.DataFrame(rows, columns=cols)
display(df)
conn.close()

,recommendation_id,recommendation_time,user_id,song_id
0,1,2025-10-01 07:42:25,2,6


In [ ]:
#2 Generate the recommendaions for Minnie
import sqlite3
import pandas as pd
from IPython.display import display

query_minnie = """
SELECT u.name AS user_name,
       s.title AS song_title,
       s.artist AS artist,
       r.recommendation_time
FROM Recommendations r
JOIN Users u ON r.user_id = u.user_id
JOIN Songs s ON r.song_id = s.song_id
WHERE u.name = 'Minnie';
"""

with sqlite3.connect(dbname) as conn:
    df = pd.read_sql_query(query_minnie, conn)

display(df)



,user_name,song_title,artist,recommendation_time
0,Minnie,Yesterday,Beatles,2025-10-01 07:42:25


In [ ]:
#3 Re-do the generation of recommendations now on the basis of listen time
import sqlite3, pandas as pd
from IPython.display import display

query_time_based = """
WITH time_listens AS (               -- only plays that have a listen_time
  SELECT user_id, song_id
  FROM Listens
  WHERE listen_time IS NOT NULL
  GROUP BY user_id, song_id
),
paired AS (                          -- co-listens within the same user
  SELECT a.user_id,
         a.song_id AS seed_song,
         b.song_id AS candidate_song
  FROM time_listens a
  JOIN time_listens b
    ON a.user_id = b.user_id
   AND a.song_id <> b.song_id
),
similar AS (                          -- item-item similarity on the time-aware set
  SELECT seed_song, candidate_song, COUNT(*) AS overlap
  FROM paired
  GROUP BY seed_song, candidate_song
  HAVING COUNT(*) > 0
),
time_recs AS (                        -- user × songs they haven’t heard (time-aware)
  SELECT DISTINCT
         tl.user_id,
         s.candidate_song AS song_id
  FROM similar s
  JOIN time_listens tl
    ON tl.song_id = s.seed_song
  WHERE s.candidate_song NOT IN (
    SELECT song_id FROM time_listens t2 WHERE t2.user_id = tl.user_id
  )
)
SELECT
  u.name  AS user_name,
  sng.title  AS song_title,
  sng.artist AS artist
FROM time_recs r
JOIN Users u  ON r.user_id = u.user_id
JOIN Songs sng ON r.song_id = sng.song_id
ORDER BY user_name, song_title;
"""

with sqlite3.connect(dbname) as conn:
    df_time = pd.read_sql_query(query_time_based, conn)

display(df_time)


,user_name,song_title,artist
0,Mickey,Hey Jude,Beatles
1,Mickey,Yellow Submarine,Beatles
2,Minnie,Yesterday,Beatles


In [ ]:
#4 Generate new recommendations
from sqlite3 import dbapi2 as sqlite3

with sqlite3.connect(dbname) as conn:
    cur = conn.cursor()
    cur.execute("PRAGMA foreign_keys = ON;")
    cur.execute("PRAGMA journal_mode = WAL;")

    # 0) Clear table (so only NEW, time-based recommendations remain)
    cur.execute("DELETE FROM Recommendations;")

    # 1) Insert time-based recommendations for all users
    sql_q4 = """
    INSERT INTO Recommendations (recommendation_id, recommendation_time, user_id, song_id)
    WITH time_listens AS (                 -- only plays that have timestamps
      SELECT user_id, song_id
      FROM Listens
      WHERE listen_time IS NOT NULL
      GROUP BY user_id, song_id
    ),
    paired AS (                            -- co-listens per user within that time-aware set
      SELECT a.user_id,
             a.song_id AS seed_song,
             b.song_id AS cand_song
      FROM time_listens a
      JOIN time_listens b
        ON a.user_id = b.user_id
       AND a.song_id <> b.song_id
    ),
    similar AS (                           -- item–item similarity (simple count)
      SELECT seed_song, cand_song, COUNT(*) AS overlap
      FROM paired
      GROUP BY seed_song, cand_song
      HAVING COUNT(*) > 0
    ),
    candidates AS (                        -- user × (unheard song) from the time-aware graph
      SELECT DISTINCT
             tl.user_id,
             s.cand_song AS song_id
      FROM similar s
      JOIN time_listens tl
        ON tl.song_id = s.seed_song
      WHERE s.cand_song NOT IN (
        SELECT song_id FROM time_listens t2 WHERE t2.user_id = tl.user_id
      )
    ),
    maxid AS ( SELECT COALESCE(MAX(recommendation_id), 0) AS m FROM Recommendations )
    SELECT
      m + ROW_NUMBER() OVER (ORDER BY user_id, song_id) AS recommendation_id,
      CURRENT_TIMESTAMP                                  AS recommendation_time,
      user_id,
      song_id
    FROM candidates, maxid;
    """
    cur.execute(sql_q4)
    conn.commit()


In [ ]:
import pandas as pd
from IPython.display import display
with sqlite3.connect(dbname) as conn:
    df = pd.read_sql_query("""
      SELECT u.name AS user_name, s.title AS song_title, s.artist, r.recommendation_time
      FROM Recommendations r
      JOIN Users u ON u.user_id = r.user_id
      JOIN Songs s ON s.song_id = r.song_id
      ORDER BY user_name, song_title;
    """, conn)
display(df)


,user_name,song_title,artist,recommendation_time
0,Mickey,Hey Jude,Beatles,2025-10-01 07:42:25
1,Mickey,Yellow Submarine,Beatles,2025-10-01 07:42:25
2,Minnie,Yesterday,Beatles,2025-10-01 07:42:25


#5. What are the differences with the static method on #2 above.

The static method looks at your entire listening history and treats every play equally whether it happened yesterday or a year ago. In contrast, the time-based method focuses on recent plays, either by using only those with timestamps or by giving more weight to newer activity. This means your most recent listening habits have a stronger impact on recommendations.

That means in practice as:

Signal Source:                                                                                                                                       
Static model: Uses all listening data without considering when it happened. Every play counts the same.
Time-based model: Prioritizes recent listens. Older plays matter less—or not at all if they lack a timestamp.

Trade-Offs:

Static can feel outdated. It may take a while to reflect new musical interests.
Time-based is more reactive. It quickly picks up on shifts in taste, but might overreact to short-term binges.

Responsiveness vs. Stability:

Static model: Offers a consistent, stable view of your preferences, but reacts slowly to change.
Time-based model: Quickly adapts to recent trends in your behavior, but can be less consistent.

Data Requirements:

Static works with minimal data. It doesn't rely on timestamps.
Time-based needs time info. It requires accurate timestamps to be effective.
Impact on Rankings:
Static favors long-term favorites. Artists you've consistently listened to rank higher.
Time-based highlights what's current. Recent discoveries and new releases are more likely to surface.
